In [ ]:
import os
import json
import requests
import csv
import numpy as np
import pandas as pd
import geopandas as gpd
from zipfile import ZipFile
from datetime import date, datetime, timedelta
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection

### Append new cases, new tested, and new hospitalizations to yesterday's data

In [ ]:
print(str(datetime.now()) + " Running 6_COVID_14day_Metrics_Process")
yearDate = date.today().strftime('%m%d%y')
print("today: "+yearDate)
yearDate_long = date.today().strftime('%m/%d/%y')
print(yearDate_long)
yesterday = date.today() - timedelta(days=1)
yearDate_yesterday = yesterday.strftime('%m%d%y')
print("yesterday: "+yearDate_yesterday)

In [ ]:
covid_cases_file = "data/Florida_COVID19_Cases_by_County{}.xlsx".format(yearDate)
df_covid_cases = pd.read_excel(covid_cases_file, sheet_name = 'Florida_COVID19_Cases_by_County')
df_covid_case = df_covid_cases.drop(columns=['Unnamed: 0'],axis=1)
df_covid_case.head()

In [ ]:
df_new_case = df_covid_cases[['County_1','NewPos']].copy()
df_new_case.head()

In [ ]:
df_new_tested = df_covid_cases[['County_1','NewTested']].copy()
df_new_tested.head()

In [ ]:
result_folder = 'results/{}'.format(yearDate)
if not os.path.exists(result_folder):
    os.makedirs(result_folder)

In [ ]:
hosp_file = os.path.join(result_folder,"FL_County_Hospital_ICU{}.xlsx".format(yearDate))
df_hosps = pd.read_excel(hosp_file, sheet_name = 'Sheet1')
df_hosp = df_hosps.drop(columns=['Unnamed: 0'],axis=1)
df_hosp.head()

In [ ]:
df_new_hosp = df_hosp[['County', 'COVID Hospitalizations']].copy()
df_new_hosp.head()

In [ ]:
covid_update_file = os.path.join('results/{}'.format(yearDate_yesterday),"COVID_daily_update{}.xlsx".format(yearDate_yesterday))
df_NewPositive0 = pd.read_excel(covid_update_file, sheet_name = 'NewPositive')
df_NewTested0 = pd.read_excel(covid_update_file, sheet_name = 'NewTested')
df_Hospitalization0 = pd.read_excel(covid_update_file, sheet_name = 'Hospitalization')

In [ ]:
# remove first unnamed column
df_Hospitalization = df_Hospitalization0.drop(columns=['Unnamed: 0'],axis=1)
df_NewPositive = df_NewPositive0.drop(columns=['Unnamed: 0'],axis=1)
df_NewTested = df_NewTested0.drop(columns=['Unnamed: 0'],axis=1)

In [ ]:
df_NewPositive.head()

In [ ]:
df_NewPositive_merge = pd.merge(df_NewPositive, df_new_case, left_on=['County_1'],right_on= ['County_1'], how = 'outer')
df_NewPositive_new = df_NewPositive_merge.rename(columns={"NewPos":yearDate_long})
df_NewPositive_new.head()

In [ ]:
df_NewTested_merge = pd.merge(df_NewTested, df_new_tested, left_on=['County_1'],right_on= ['County_1'], how = 'outer')
df_NewTested_new = df_NewTested_merge.rename(columns={"NewTested":yearDate_long})
df_NewTested_new.head()

In [ ]:
df_Hospitalization_merge = pd.merge(df_Hospitalization, df_new_hosp, left_on=['County'],right_on= ['County'], how='outer')
df_Hospitalization_new = df_Hospitalization_merge.rename(columns={"COVID Hospitalizations":yearDate_long})
df_Hospitalization_new.head()

In [ ]:
# Export updated covid data to excel file
print("Saving COVID_daily_update{}.xlsx in results/".format(yearDate))
with pd.ExcelWriter(os.path.join(result_folder,'COVID_daily_update{}.xlsx'.format(yearDate))) as writer:  
    df_NewPositive_new.to_excel(writer, sheet_name='NewPositive')
    df_NewTested_new.to_excel(writer, sheet_name='NewTested')
    df_Hospitalization_new.to_excel(writer, sheet_name='Hospitalization')

### Calculate 7 days, 14 days metrics

In [ ]:
df_NewPositive_new['7dayPos'] = df_NewPositive_new.iloc[:,-7:].sum(axis=1)
df_NewPositive_new['14dayPos'] = df_NewPositive_new.iloc[:,-15:-1].sum(axis=1)
df_NewPositive_new

In [ ]:
df_NewTested_new['7dayTested'] = df_NewTested_new.iloc[:,-7:].sum(axis=1)
df_NewTested_new['14dayTested'] = df_NewTested_new.iloc[:,-15:-1].sum(axis=1)
df_NewTested_new

In [ ]:
df_Hospitalization_new.iloc[:,-1]

In [ ]:
df_Hospitalization_new['NewHosp14AVE'] = (df_Hospitalization_new.iloc[:,-1] - df_Hospitalization_new.iloc[:,-15])/14
df_Hospitalization_new['Hosp14AVE'] = df_Hospitalization_new.iloc[:,-15:-1].mean(axis=1)
df_Hospitalization_new['Hosp7AVE'] = df_Hospitalization_new.iloc[:,-9:-2].mean(axis=1)
df_Hospitalization_new.head()

In [ ]:
df_case_merge = pd.merge(df_NewPositive_new,df_NewTested_new, left_on = ['County_1'], right_on = ['County_1'], how="outer")
df_case_merge

In [ ]:
df_case = df_case_merge[['DEPCODE_x','County_1','COUNTYNAME_x','7dayPos','14dayPos','7dayTested','14dayTested']].copy()
df_case

In [ ]:
df_case['7daysPRate']=df_case['7dayPos']/df_case['7dayTested']
df_case['14daysPRate']=df_case['14dayPos']/df_case['14dayTested']
df_case['WeekPR_Ratio']=df_case['7daysPRate']/((df_case['14dayPos']-df_case['7dayPos'])/(df_case['14dayTested']-df_case['7dayTested']))
df_case['WeekPosi_Ratio']=df_case['7dayPos']/(df_case['14dayPos']-df_case['7dayPos'])

In [ ]:
df_case 

In [ ]:
df_hosp_14 = df_Hospitalization_new[['County','NewHosp14AVE','Hosp14AVE','Hosp7AVE']].copy()
df_hosp_14new = df_hosp_14.astype({'NewHosp14AVE': 'int64','Hosp14AVE': 'int64'})
df_hosp_14new.head()

In [ ]:
a = set(df_case['COUNTYNAME_x'].values)
b = set(df_hosp_14new['County'].values)

In [ ]:
b.difference(a)

In [ ]:
df_hosp_14new.loc[df_hosp_14new['County']=='MIAMI-DADE','County']='DADE'

In [ ]:
a = set(df_case['COUNTYNAME_x'].values)
b = set(df_hosp_14new['County'].values)
b.difference(a)

In [ ]:
df_merge = pd.merge(df_case, df_hosp_14new, left_on=['COUNTYNAME_x'],right_on= ['County'], how = 'left')
df_merge['WeekHosp_Ratio']=df_merge['Hosp7AVE']/(df_merge['Hosp14AVE']*14-df_merge['Hosp7AVE']*7)*7
df_merge.head(len(df_merge))

In [ ]:
df_sort = df_merge.sort_values(by=['DEPCODE_x'])
df_sort['County_1']=df_sort['County_1'].str.title()
df_sort

In [ ]:
df_sort_extract = df_sort.drop(columns=['County'])
df_sort_drop_unknown = df_sort_extract.drop([68])
df_sort_drop_unknown

In [ ]:
print("Saving COVID_14_update{}.xlsx in results/".format(yearDate))
df_sort_drop_unknown.to_excel(os.path.join(result_folder,"COVID_14_update{}.xlsx".format(yearDate)))

### Generate shapefile

In [ ]:
gdf_county_polygon = gpd.read_file("shp/Florida_Counties.shp")
gdf_county_polygon.head()

In [ ]:
a = set(df_sort_drop_unknown['County_1'].values)
b = set(gdf_county_polygon['COUNTYNAME'].values)
a.difference(b)

In [ ]:
b.difference(a)

In [ ]:
df_merge_poly = pd.merge(df_sort_drop_unknown, gdf_county_polygon, left_on = ['County_1'],right_on= ['COUNTYNAME'], how = 'outer')
df_merge_poly.head(len(df_merge_poly))

In [ ]:
df_merge_poly_drop = df_merge_poly.drop(columns=['DEPCODE','COUNTYNAME','COUNTY','OBJECTID','ShapeSTAre','ShapeSTLen','DATESTAMP'])
df_merge_poly_clean = df_merge_poly_drop.rename(columns={'DEPCODE_x':'DEPCODE','County_1':'County','COUNTYNAME_x':'COUNTYNAME'})
df_merge_poly_clean

In [ ]:
shpfile_name = "covid_14_metrics{}".format(yearDate)
print("Saving {}.shp in results/".format(shpfile_name))
gdf_merge_poly = gpd.GeoDataFrame(df_merge_poly_clean, geometry='geometry',crs="EPSG:4326")
gdf_merge_poly.to_file(driver = 'ESRI Shapefile', filename= os.path.join(result_folder,"{}.shp".format(shpfile_name)))

In [ ]:
print("Saving {}.zip in publish/".format(shpfile_name))
with ZipFile(os.path.join('publish', '{}.zip'.format(shpfile_name)), 'w') as zipObj:
    for file in os.listdir(result_folder):
        if shpfile_name in file:
            file_path = os.path.join(result_folder, file)
            print(file)
            try:
                zipObj.write(file_path, os.path.basename(file_path))
            except OSError as err:
                print("OS error: {0}".format(err))   

### Update the AGOL layer with the shapefile

In [ ]:
# Check if the data is updated or not today
yesterday_covid_file = "data/Florida_COVID19_Cases_by_County{}.xlsx".format(yearDate_yesterday)
today_covid_file = "data/Florida_COVID19_Cases_by_County{}.xlsx".format(yearDate)
df_yester_covid = pd.read_excel(yesterday_covid_file, sheet_name = 'Florida_COVID19_Cases_by_County')
df_today_covid = pd.read_excel(today_covid_file, sheet_name = 'Florida_COVID19_Cases_by_County')
df_yester_newpos = df_yester_covid[['County_1','NewPos']]
df_today_newpos = df_today_covid[['County_1','NewPos']]

In [ ]:
if df_yester_newpos.equals(df_today_newpos): 
    raise Exception("Today's data is not updated yet. Exit the script.")
else: 
    print ("Today's data is updated.") 

In [ ]:
print("Updating COVID 14 days Health Metrics layer")
uflURL = "https://ufl.maps.arcgis.com/"
with open("env.json", "r") as f:
    tokenfile = json.load(f)

In [ ]:
my_token = tokenfile['token']
AGOLConnection = GIS(token=my_token)
print(AGOLConnection.users.__dict__)

In [ ]:
covid14_zip_file = "publish/covid_14_metrics{}.zip".format(yearDate)
covid14_content = AGOLConnection.content.get('e59aafcbc5bc4287ab0351abbdee45fb')
covid14_content

In [ ]:
covid14_collection = FeatureLayerCollection.fromitem(covid14_content)
response = covid14_collection.manager.overwrite(covid14_zip_file)
print(response)

In [ ]:
print(str(datetime.now()) + " Done with 6_COVID_14day_Metrics_Process")